# Sequence chat - extract data, draw image, and summarize




## Import libaries and prepare the API credential

In [1]:
import tempfile
import shutil
import os
import subprocess

from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

with open('openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o'

## Create a temporary folder to execute code


In [2]:
temp_dir = tempfile.TemporaryDirectory()
print(temp_dir.name)

subprocess.run(["cp", "population.csv", temp_dir.name+"/"])

os.listdir(temp_dir.name)

/tmp/tmpwvuvxicb


['population.csv']

## Agent 1: code executor, who can run python code locally

In [3]:
executor = LocalCommandLineCodeExecutor(
    timeout=10,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

# Create an agent with code executor configuration.
code_executor = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"executor": executor},  # Use the local command line code executor.
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

## Agent 2: data scientist, who can write python code

In [4]:
data_scientist_system_message = """You are a data scientist, and you can solve problems by writing python code. 
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
Some of the tasks are about analyzing a csv file. If you don't know the structure of the csv file, you may provide code to the users to extract the csv structure, and use the feedback from the users to generate the csv analysis code. 
"""

data_scientist = ConversableAgent(
    "data_scientist_agent",
    system_message = data_scientist_system_message,
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False,  # Turn off code execution for this agent.
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

In [ ]:

def my_summary_method(
    sender: ConversableAgent,
    recipient: ConversableAgent,
    summary_args: dict,
):
    return sender.chat_messages[recipient][-2]["content"]

chat_results = code_executor.initiate_chats(
    [
        {
            "recipient": data_scientist,
            "message": "There is a csv file called population.csv, can you find out the number of newborns each year for Norway, and store it in a result.csv file",
            "max_turns": 2,
            "summary_method": my_summary_method
        },
        {
            "recipient": data_scientist,
            "message": "the previous task produced a csv file, could you draw a bar chart for this result, in a result.png file, and print out the content of the csv file in the console",
            "max_turns": 2,
            "summary_method": my_summary_method
        }
    ]
)

os.listdir(temp_dir.name)


********************************************************************************
Starting a new chat....

********************************************************************************
code_executor_agent (to data_scientist_agent):

There is a csv file called population.csv, can you find out the number of newborns each year for Norway, and store it in a result.csv file

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_scientist_agent (to code_executor_agent):

Sure, the plan is as follows:

1. **Read the `population.csv` file** to understand its structure.
2. **Process the data** to extract the number of newborns each year for Norway.
3. **Save the extracted data** into a new file called `result.csv`.

Let's start by reading the `population.csv` file to understand its contents. This will allow us to tailor the analysis to the structure of the data.

```python
# filename: read_population.py

import pandas as pd

# Rea

/opt/conda/lib/python3.11/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Provide feedback to data_scientist_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to data_scientist_agent):

exitcode: 0 (execution succeeded)
Code output:   Country name  ...  population__all__95_99__records
0  Afghanistan  ...                                8
1  Afghanistan  ...                                9
2  Afghanistan  ...                               10
3  Afghanistan  ...                               11
4  Afghanistan  ...                               11

[5 rows x 40 columns]


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
data_scientist_agent (to code_executor_agent):

Based on the output, it seems the CSV file has detailed population records, possibly across different age groups or categories. To find the number of newborns each year for Norway, we need to identify columns related to newborns and isolate the relevant data for Norway. Let

In [ ]:
chart = "result.png"
from PIL import Image
from IPython.display import display
img = Image.open(temp_dir.name + "/" + chart)
display(img)